# Goal

The goal is to gain insights into how the company can cultivate more lifelong customers, defined in this context as clients who order almost every month. 
I aim to test the hypothesis regarding whether shipping costs influence the ordering behavior of customers.

# Importing modules, libraries, data

In [ ]:
# code to ensure that any changes made to the functions.py file are reflected in the Jupyter Notebook upon import
%load_ext autoreload
%autoreload 2

In [ ]:
# importing modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import functions
from functions import (
    cleaning,
    formatting,
    new_columns,
    fuzzy_match_clients,
    anonymize_clients,
    dropping_column_with_private_info
)

# For privacy reasons this functions_clients file will not be shared
import functions_clients
from functions_clients import (
    cleaning_invoice_number,
    cleaning_private_info,
    deleting_test_orders
)

from thefuzz import fuzz

In [ ]:
df = pd.read_csv("orders_data.csv")

# Data Cleaning

In [ ]:
# Data cleaning
df = cleaning(df)

# Formatting dataframe, like datetime, column names etc. 
df = formatting(df)

# New columns that will be helpful for the analysis: "Customer full address" and "Subtotal products".
df = new_columns(df)


# Functions for cleaning client info
# For privacy reasons these functions will not be shared
df = cleaning_invoice_number(df)
df = cleaning_private_info(df)
df = deleting_test_orders(df)


# Identifing and correcting similar client information
df = fuzzy_match_clients(df)

# Creating a column with a code for each unique client
df = anonymize_clients(df)

# Deleting the columns that contain private info of clients
df = dropping_column_with_private_info(df)

# Resetting index
df = df.reset_index(drop=True)

# Analysis

In [ ]:
# Creating a new column that has a format like: Mar 2023
df["Order date by month"] = df["Order date"].dt.strftime('%b %Y') 
df["Order date by month"]

# Taking data only from the last 2 years, to keep the data more relevant
# Because I will examine whole months, I am taking until Feb in consideration (currently it is mid March)
current_date = pd.Timestamp.now()

two_years_data = df[(df['Order date'] >= current_date - pd.DateOffset(years=2)) & 
                    (df['Order date'] < pd.Timestamp(year=current_date.year, month=current_date.month, day=1))]

sales_per_month = two_years_data.groupby('Order date by month')['Subtotal products'].sum()

# Convert 'Order date by month' to categorical with preferred order
preferred_month_order = ['Mar 2022', 'Apr 2022', 'May 2022', 'Jun 2022', 'Jul 2022', 'Aug 2022', 'Sep 2022', 
                         'Oct 2022', 'Nov 2022', 'Dec 2022', 'Jan 2023', 'Feb 2023', 'Mar 2023', 'Apr 2023', 
                         'May 2023', 'Jun 2023', 'Jul 2023', 'Aug 2023', 'Sep 2023', 
                         'Oct 2023', 'Nov 2023', 'Dec 2023', 'Jan 2024', 'Feb 2024']

### Sales in the last 2 years

In [ ]:
# PLOT to show increase of sales the last 2 years

# Sort the data based on the preferred month order
sales_per_month = sales_per_month.reindex(preferred_month_order)

# Plot the data as a line graph
sales_per_month.plot(figsize=(10, 6), color='#E92D7A', linestyle='-',linewidth=10)

# Adding labels and title
plt.title('Increase of sales over the last 2 years (monthly)', fontsize=20)
plt.xlabel('Date', fontsize=18)
plt.ylabel('Sales amount', fontsize=18)
plt.grid(True)

# Adjust font size of ticks
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.savefig('sales_over_last_two_years.png')

# Displaying the plot
plt.show()

### Orders in the last 2 years

In [ ]:
orders_per_month = two_years_data.groupby('Order date by month')['Invoice number'].count()

# Sort the data based on the preferred month order
orders_per_month = orders_per_month.reindex(preferred_month_order)

# Plot the data as a line graph
orders_per_month.plot(figsize=(10, 6), color='#E92D7A', linestyle='-',linewidth=10)

# Adding labels and title
plt.title('Increase of orders over the last 2 years (monthly)', fontsize=20)
plt.xlabel('Date', fontsize=18)
plt.ylabel('Orders', fontsize=18)
plt.grid(True)

# Adjust font size of ticks
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.savefig('orders_over_last_two_years.png')

# Displaying the plot
plt.show()

### Average order value in the last 2 years

In [ ]:
# Group by 'Order date by month' and calculate sum of 'Subtotal products' and count of 'Invoice number'
monthly_summary = two_years_data.groupby('Order date by month').agg({'Subtotal products': 'sum', 'Invoice number': 'count'})

# Calculate the average order value per month
avg_order_value_per_month = monthly_summary['Subtotal products'] / monthly_summary['Invoice number']

# Sort the data based on the preferred month order
avg_order_value_per_month = avg_order_value_per_month.reindex(preferred_month_order)

# Plot the data as a line graph
plt.figure(figsize=(10, 6))
avg_order_value_per_month.plot(color='#E92D7A', linestyle='-',linewidth=10)

# Adding labels and title
plt.title('Average order value over the last 2 years (monthly)', fontsize=20)
plt.xlabel('Month', fontsize=18)
plt.ylabel('Average Order Value', fontsize=18)
plt.grid(True)

# Adjust font size of ticks
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.savefig('average_order_value_over_last_two_years.png')

# Displaying the plot
plt.show()

### Order behaviour of clients in the last 1 year - Order Frequency

In [ ]:
# Taking data only from the last year, to keep the data more relevant, examinining only whole months
last_year_data = df[(df['Order date'] >= current_date - pd.DateOffset(years=1)) & 
                    (df['Order date'] < pd.Timestamp(year=current_date.year, month=current_date.month, day=1))]

order_frequency = last_year_data.groupby(["Client codes","Order date by month"])["Invoice number"].count()

In [ ]:
# resetting index to convert the DataFrameGroupBy object to dataframe
order_frequency = order_frequency.reset_index()

# Create a pivot table to indicate whether a client has ordered for each month
pivot_table = order_frequency.pivot_table(index='Client codes', columns='Order date by month', values='Invoice number', aggfunc='count', fill_value=0)

# Total sum of months with order
months_ordered = pivot_table.sum(axis=1)

def label_map(num):
    if num < 2:
        return "1 month"
    elif num < 6:
        return "2-5 months"
    elif num < 11:
        return "6-10 months"
    else:
        return "Every month (11-12 months)"

# applying label map and value counts of the labels
# so here I have each client of the last year, and the label of them (1 month, 2-5 months etc)
# I will use that also below for other plots/ analysis
ordering_behavior_labels = months_ordered.apply(label_map)
ordering_behavior_counts = ordering_behavior_labels.value_counts()

# Reindexing to have the preferred ordered
ordered_index = ['1 month', '2-5 months', '6-10 months', 'Every month (11-12 months)']
ordering_behavior_counts = ordering_behavior_counts.reindex(ordered_index)

colors = ['#009FE3', '#99C331', '#F56C22', '#A353DB']

# Create a bar plot
plt.figure(figsize=(8, 6))
ordering_behavior_counts.plot(kind='bar', color=colors)

# Adding labels and title
plt.title('Ordering Behavior of Clients (Last 1 Year)')
plt.xlabel('Ordering Behavior')
plt.ylabel('Number of Clients')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability

plt.savefig('ordering_behaviour_of_clients.png', bbox_inches='tight')

# Display the plot
plt.show()

### Order behaviour of clients in the last 1 year - Average Order Amount

In [ ]:
# here I take the first() cause I know that there is only one value per invoice number
average_subtotal_frequency = last_year_data.groupby(["Client codes","Order date by month",
                                                   "Invoice number"])["Subtotal products"].first()

# resetting index to convert the DataFrameGroupBy object to dataframe
average_subtotal_frequency = average_subtotal_frequency.reset_index()

# Create a pivot table to indicate whether a client has ordered for each month
pivot_table_2 = average_subtotal_frequency.pivot_table(index='Client codes', 
                                                    columns='Order date by month', 
                                                    values='Subtotal products', 
                                                    aggfunc= lambda x: x.mean().round(2), 
                                                    fill_value=0)

# making dataframe with client codes and their average subtotal per order for the year
average_subtotal = pivot_table_2.apply(lambda x: x[x != 0.0].mean(), axis=1)

# here I replace the client codes with their label (of how often they order) that I have generated above
average_subtotal.index = ordering_behavior_labels.values

# Group the average_subtotal Series by its index and find the mean for each unique index value.
average_subtotal_counts = average_subtotal.groupby(level=0).mean()

# Reindexing to have the preferred ordered
ordered_index = ['1 month', '2-5 months', '6-10 months', 'Every month (11-12 months)']
average_subtotal_counts = average_subtotal_counts.reindex(ordered_index)

colors = ['#009FE3', '#99C331', '#F56C22', '#A353DB']

# Plot the average subtotal products
plt.figure(figsize=(8, 6))
average_subtotal_counts.plot(kind='bar', color=colors)

# Adding labels and title
plt.title('Average order amount by Ordering Behavior (Last 1 Year)')
plt.xlabel('Ordering Behavior')
plt.ylabel('Average Order Amount')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability

plt.savefig('average_subtotal_products_by_ordering_behavior.png', bbox_inches='tight')
plt.show()

### Order behaviour of clients in the last 1 year - Shipping costs

In [ ]:
# doing the same process as above for Shipping fees

# here I take the first() cause I know that there is only one value per invoice number
average_shipping_fees_frequency = last_year_data.groupby(["Client codes","Order date by month",
                                                   "Invoice number"])["Shipping fees"].first()

# resetting index to convert the DataFrameGroupBy object to dataframe
average_shipping_fees_frequency = average_shipping_fees_frequency.reset_index()

# Create a pivot table to indicate whether a client has ordered for each month
pivot_table_3 = average_shipping_fees_frequency.pivot_table(index='Client codes', 
                                                    columns='Order date by month', 
                                                    values='Shipping fees', 
                                                    aggfunc= lambda x: x.mean().round(2), 
                                                    fill_value=0)

# making dataframe with client codes and their average shipping fees per order for the year
average_shipping_fees = pivot_table_3.apply(lambda x: x[x != 0.0].mean(), axis=1)

# here I replace the client codes with their label (of how often they order) that I have generated above
average_shipping_fees.index = ordering_behavior_labels.values

# Group the average shipping fees Series by its index and find the mean for each unique index value.
average_shipping_fees_counts = average_shipping_fees.groupby(level=0).mean()

# Reindexing to have the preferred ordered
ordered_index = ['1 month', '2-5 months', '6-10 months', 'Every month (11-12 months)']
average_shipping_fees_counts = average_shipping_fees_counts.reindex(ordered_index)

colors = ['#009FE3', '#99C331', '#F56C22', '#A353DB']

# Plot the average shipping costs
plt.figure(figsize=(8, 6))
average_shipping_fees_counts.plot(kind='bar', color=colors)

# Adding labels and title
plt.title('Average Shipping Costs by Ordering Behavior (Last 1 Year)')
plt.xlabel('Ordering Behavior')
plt.ylabel('Average Shipping Costs')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability

plt.savefig('average_shipping_costs_by_ordering_behavior.png', bbox_inches='tight')
plt.show()